## Image Generation

In [ ]:
# !pip install transformers==4.49 diffusers==0.32.2
# !wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/utils.py
# !wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/LandscapeGenerator.py

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = ""

from datasets import load_dataset
from PIL import Image as PImage

from LandscapeGenerator import LandscapeGenerator

In [ ]:
newsdata_ds = load_dataset("thiagohersan/newsdata-images", split="newsdata")
newsdata_data = [img for img in newsdata_ds]
newsdata_data_date = sorted(newsdata_data, key=lambda x: x["pubDate"], reverse=True)[:64]

In [ ]:
models = [
  "runwayml/stable-diffusion-inpainting",
  "stable-diffusion-v1-5/stable-diffusion-inpainting",
  "stabilityai/stable-diffusion-2-inpainting"
]

mLG = LandscapeGenerator(newsdata_data_date, models[1])

### New landscape

In [ ]:
mLG.gen_landscape(keep_width=256, size=(1440, 512), n=32, label="20251023_00")

### Continue with seed

In [ ]:
seed = PImage.open("./imgs/20251023_04/20251023_04_32.jpg")
mLG.gen_landscape(keep_width=256, size=(1440, 512), n=32, label="20251023_05", seed_img=seed)

### Graft from single image

In [ ]:
limg = PImage.open("./imgs/20251023_03/20251023_03_15.jpg")
label = "20251023_03_15"

keep_width=500
limg.crop((0,0,keep_width,512)).save(f"./imgs/foo/{label}a.jpg")

i,m = LandscapeGenerator.add_crop(limg, 0, keep_width, (1440,512), None, 0)
img_in,mask_in = LandscapeGenerator.add_crop(limg, 600, 1500, i, m, 880)
display(img_in)
display(mask_in)

### Graft from two image

In [ ]:
limg = PImage.open("./imgs/20251023_02/20251023_02_11.jpg")
rimg = PImage.open("./imgs/20251023_02/20251023_02_12.jpg")
label = "20251023_02_12"
# display(limg)
# display(rimg)

keep_width=256
img_in,mask_in = mLG.prep_graft(limg, rimg, keep_width=keep_width, right_offset=500, label=label)

display(img_in)
# display(mask_in)

In [ ]:
output = mLG.pipe(
  prompt="edge of rolling river",
  negative_prompt="repetitive, stretched, vertical splits, sharp transitions",
  image=img_in,
  mask_image=mask_in,
  width=img_in.size[0], height=img_in.size[1],
  guidance_scale=7.5,
  strength=1.0,
  num_inference_steps=32,
  num_images_per_prompt=4,
)

for i in output.images:
  display(i)

In [ ]:
iidx = 1
iw,ih = output.images[iidx].size
output.images[iidx].crop((keep_width,0,iw,ih)).save(f"imgs/{label}a.jpg")

In [ ]:
LandscapeGenerator.stitch_images("./imgs/20251022_06")